In [1]:
import emoji
import numpy as np
import pandas as pd

from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import SimpleRNN,LSTM,Dense,Activation,Input,Dropout


Using TensorFlow backend.


In [22]:
train_data = pd.read_csv('./train_emoji.csv',header = None)
test_data = pd.read_csv('./test_emoji.csv',header = None)

In [23]:
train_data.head()

,0,1,2,3
0,never talk to me again,3,NaN,NaN
1,I am proud of your achievements,2,NaN,NaN
2,It is the worst day in my life,3,NaN,NaN
3,Miss you so much,0,NaN,[0]
4,food is life,4,NaN,NaN


In [24]:
test_data.head()

,0,1
0,I want to eat\t,4
1,he did not answer\t,3
2,he got a raise\t,2
3,she got me a present\t,0
4,ha ha ha it was so funny\t,2


In [25]:
x_train = train_data.values
y_train = x_train[:,1]
x_train = x_train[:,0]
y_train = np_utils.to_categorical(y_train)
print(x_train.shape,y_train.shape)

x_test = test_data.values
y_test = x_test[:,1]
y_act = x_test[:,1]
x_test = x_test[:,0]
y_test = np_utils.to_categorical(y_test)
print(x_test.shape,y_test.shape)


(132,) (132, 5)
(56,) (56, 5)


In [6]:
emoji_dict = { 0 : ":heart:", 1 : ":baseball:", 2 : ":smile:", 3 : ":disappointed:", 4 : ":fork_and_knife:"}
for ix in emoji_dict.keys():
    print(ix,emoji.emojize(emoji_dict[ix],use_aliases=True))

0 ❤
1 ⚾
2 😄
3 😞
4 🍴


In [7]:
for ix in range(x_train.shape[0]):
    x_train[ix] = x_train[ix].split()
    
for ix in range(x_test.shape[0]):
    x_test[ix] = x_test[ix].split()
    

In [9]:
embedding_index = {}

f = open('./glove.6B.50d.txt')

for line in f:
    data = line.split()
    word = data[0]
    coeff = np.asarray(data[1:], dtype='float32')
    embedding_index[word] = coeff
f.close()
print(len(embedding_index))
    

400000


In [10]:
print(embedding_index['india'].shape)

(50,)


In [11]:
max_length_train = np.sort(np.array([len(ix) for ix in x_train]))[-1]
print(max_length_train)

embedding_matrix_train = np.zeros((x_train.shape[0], max_length_train, 50))
embedding_matrix_test = np.zeros((x_test.shape[0], max_length_train, 50))

for ix in range(x_train.shape[0]):
    for ij in range(len(x_train[ix])):
        embedding_matrix_train[ix][ij] = embedding_index[x_train[ix][ij].lower()]
        
for ix in range(x_test.shape[0]):
    for ij in range(len(x_test[ix])):
        embedding_matrix_test[ix][ij] = embedding_index[x_test[ix][ij].lower()]  
print(embedding_matrix_train.shape, embedding_matrix_test.shape)

10
(132, 10, 50) (56, 10, 50)


# RNN MODEL

In [12]:
model = Sequential()
model.add(SimpleRNN(64, input_shape=(10,50), return_sequences=True))
model.add(Dropout(0.5))
model.add(SimpleRNN(64, return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_1 (SimpleRNN)     (None, 10, 64)            7360      
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 64)            0         
_________________________________________________________________
simple_rnn_2 (SimpleRNN)     (None, 64)                8256      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 325       
_________________________________________________________________
activation_1 (Activation)    (None, 5)                 0         
Total params: 15,941
Trainable params: 15,941
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
hist = model.fit(embedding_matrix_train,y_train,epochs = 60, batch_size=32,shuffle=True,validation_data = (embedding_matrix_test,y_test))

Train on 132 samples, validate on 56 samples
Epoch 1/60
132/132 [==============================] - 1s 10ms/step - loss: 1.9594 - acc: 0.2273 - val_loss: 1.4924 - val_acc: 0.3571
Epoch 2/60
132/132 [==============================] - 0s 194us/step - loss: 1.8103 - acc: 0.2652 - val_loss: 1.4961 - val_acc: 0.3393
Epoch 3/60
132/132 [==============================] - 0s 212us/step - loss: 1.5802 - acc: 0.3561 - val_loss: 1.4678 - val_acc: 0.3929
Epoch 4/60
132/132 [==============================] - 0s 265us/step - loss: 1.6381 - acc: 0.3409 - val_loss: 1.4655 - val_acc: 0.3929
Epoch 5/60
132/132 [==============================] - 0s 235us/step - loss: 1.5768 - acc: 0.3106 - val_loss: 1.4635 - val_acc: 0.3929
Epoch 6/60
132/132 [==============================] - 0s 273us/step - loss: 1.4622 - acc: 0.3788 - val_loss: 1.4463 - val_acc: 0.3571
Epoch 7/60
132/132 [==============================] - 0s 220us/step - loss: 1.4566 - acc: 0.4167 - val_loss: 1.4179 - val_acc: 0.3571
Epoch 8/60
132/132

In [26]:
pred = model.predict_classes(embedding_matrix_test)

float(sum(pred==y_act))/embedding_matrix_test.shape[0]

0.5535714285714286

# LSTM MODEL

In [27]:
model = Sequential()
model.add(LSTM(128, input_shape=(10,50), return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(128, return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 10, 128)           91648     
_________________________________________________________________
dropout_3 (Dropout)          (None, 10, 128)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 645       
_________________________________________________________________
activation_2 (Activation)    (None, 5)                 0         
Total params: 223,877
Trainable params: 223,877
Non-trainable params: 0
_________________________________________________________________


In [28]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
hist = model.fit(embedding_matrix_train,y_train,epochs = 50, batch_size=32,shuffle=True,validation_data = (embedding_matrix_test,y_test))

Train on 132 samples, validate on 56 samples
Epoch 1/50
132/132 [==============================] - 1s 9ms/step - loss: 1.5972 - acc: 0.2500 - val_loss: 1.5324 - val_acc: 0.3214
Epoch 2/50
132/132 [==============================] - 0s 866us/step - loss: 1.5509 - acc: 0.2955 - val_loss: 1.5120 - val_acc: 0.2857
Epoch 3/50
132/132 [==============================] - 0s 841us/step - loss: 1.4876 - acc: 0.3106 - val_loss: 1.4637 - val_acc: 0.3393
Epoch 4/50
132/132 [==============================] - 0s 811us/step - loss: 1.4314 - acc: 0.3485 - val_loss: 1.4184 - val_acc: 0.3750
Epoch 5/50
132/132 [==============================] - 0s 837us/step - loss: 1.3583 - acc: 0.4394 - val_loss: 1.3508 - val_acc: 0.4464
Epoch 6/50
132/132 [==============================] - 0s 855us/step - loss: 1.2147 - acc: 0.5985 - val_loss: 1.2677 - val_acc: 0.5179
Epoch 7/50
132/132 [==============================] - 0s 783us/step - loss: 1.0747 - acc: 0.6212 - val_loss: 1.1847 - val_acc: 0.4643
Epoch 8/50
132/132 

In [30]:
pred = model.predict_classes(embedding_matrix_test)

float(sum(pred==y_act))/embedding_matrix_test.shape[0]

0.6428571428571429

# Both combined

In [31]:
model = Sequential()
model.add(SimpleRNN(64, input_shape=(10,50), return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(64, return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_3 (SimpleRNN)     (None, 10, 64)            7360      
_________________________________________________________________
dropout_5 (Dropout)          (None, 10, 64)            0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dropout_6 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 325       
_________________________________________________________________
activation_3 (Activation)    (None, 5)                 0         
Total params: 40,709
Trainable params: 40,709
Non-trainable params: 0
_________________________________________________________________


In [32]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
hist = model.fit(embedding_matrix_train,y_train,epochs = 100, batch_size=32,shuffle=True,validation_data = (embedding_matrix_test,y_test))

Train on 132 samples, validate on 56 samples
Epoch 1/100
132/132 [==============================] - 1s 7ms/step - loss: 1.6155 - acc: 0.2424 - val_loss: 1.5608 - val_acc: 0.2500
Epoch 2/100
132/132 [==============================] - 0s 381us/step - loss: 1.5564 - acc: 0.3106 - val_loss: 1.5422 - val_acc: 0.3036
Epoch 3/100
132/132 [==============================] - 0s 387us/step - loss: 1.5378 - acc: 0.2955 - val_loss: 1.5224 - val_acc: 0.2857
Epoch 4/100
132/132 [==============================] - 0s 394us/step - loss: 1.4767 - acc: 0.3485 - val_loss: 1.4913 - val_acc: 0.4286
Epoch 5/100
132/132 [==============================] - 0s 460us/step - loss: 1.4874 - acc: 0.3409 - val_loss: 1.4542 - val_acc: 0.4107
Epoch 6/100
132/132 [==============================] - 0s 491us/step - loss: 1.4050 - acc: 0.4394 - val_loss: 1.4089 - val_acc: 0.4464
Epoch 7/100
132/132 [==============================] - 0s 539us/step - loss: 1.3711 - acc: 0.3788 - val_loss: 1.3721 - val_acc: 0.4821
Epoch 8/100


Epoch 61/100
132/132 [==============================] - 0s 485us/step - loss: 0.0525 - acc: 0.9773 - val_loss: 1.6206 - val_acc: 0.6964
Epoch 62/100
132/132 [==============================] - 0s 446us/step - loss: 0.0453 - acc: 0.9848 - val_loss: 1.5532 - val_acc: 0.6607
Epoch 63/100
132/132 [==============================] - 0s 448us/step - loss: 0.0223 - acc: 1.0000 - val_loss: 1.4977 - val_acc: 0.6786
Epoch 64/100
132/132 [==============================] - 0s 422us/step - loss: 0.0201 - acc: 1.0000 - val_loss: 1.5041 - val_acc: 0.6607
Epoch 65/100
132/132 [==============================] - 0s 446us/step - loss: 0.0228 - acc: 0.9924 - val_loss: 1.5304 - val_acc: 0.6607
Epoch 66/100
132/132 [==============================] - 0s 389us/step - loss: 0.0153 - acc: 1.0000 - val_loss: 1.5485 - val_acc: 0.6429
Epoch 67/100
132/132 [==============================] - 0s 405us/step - loss: 0.0181 - acc: 1.0000 - val_loss: 1.5455 - val_acc: 0.6607
Epoch 68/100
132/132 [==========================

In [33]:
pred = model.predict_classes(embedding_matrix_test)

float(sum(pred==y_act))/embedding_matrix_test.shape[0]

0.7142857142857143